In [1]:
import pandas as pd
import os
import requests
import json
from IPython.display import JSON
import gmplot

In [59]:
# Yelp API pull
url = 'https://api.yelp.com/v3/businesses/search'
YELP_APIKEY=os.environ['YELP_APIKEY']

class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r 

In [85]:
def get_business(loc,terms):
    params={
        'location':loc,
        'radius':10000,
        'price':1,
        'term':terms,
        'categories':'Food',
        'limit':50,
        'sort_by':'rating'
        }
    response=requests.get(url,params=params,auth=BearerAuth(YELP_APIKEY))
    return response.json()

In [86]:
bby_snack_yelp = get_business('49.225706, -123.001991','snack') #metrotown 
rmd_snack_yelp = get_business('49.156481, -123.143042','snack') #blundell 
van_dt_snack_yelp = get_business('49.285766, -123.130353','snack') #west-end 
van_west_snack_yelp = get_business('49.241093, -123.140106','snack') #Shaughnessy

In [2]:
import pickle

In [88]:
pickle.dump(bby_snack_yelp,open('bby_snack_yelp.txt','wb'))
pickle.dump(rmd_snack_yelp,open('rmd_snack_yelp.txt','wb'))
pickle.dump(van_dt_snack_yelp,open('van_dt_snack_yelp.txt','wb'))
pickle.dump(van_west_snack_yelp,open('van_west_snack_yelp.txt','wb'))

In [3]:
bby_snack_yelp=pickle.load(open('bby_snack_yelp.txt','rb'))
rmd_snack_yelp=pickle.load(open('rmd_snack_yelp.txt','rb'))
van_dt_snack_yelp=pickle.load(open('van_dt_snack_yelp.txt','rb'))
van_west_snack_yelp=pickle.load(open('van_west_snack_yelp.txt','rb'))

In [4]:
JSON(van_west_snack_yelp)

<IPython.core.display.JSON object>

In [5]:
responses=[bby_snack_yelp,rmd_snack_yelp,van_dt_snack_yelp,van_west_snack_yelp]
area=['bby','rmd','van_dt','van_west']

In [6]:
business={
    'name':[],
    'loc':[],
    'rating':[],
    'price':[],
    'area':[],
    'categories':[]
}

for ind in range(len(responses)):
    a=area[ind]
    response=responses[ind]
    for i in range(len(response['businesses'])):
        business['name'].append(response['businesses'][i]['name'])
        business['loc'].append((
            response['businesses'][i]['coordinates']['latitude'],
            response['businesses'][i]['coordinates']['longitude']))
        business['rating'].append(response['businesses'][i]['rating'])
        business['area'].append(a)
        try:
            business['price'].append(response['businesses'][i]['price'])
        except:
            business['price'].append('na')
        try:
            types=response['businesses'][i]['categories']
            res=[]
            for i in range(len(types)):
                res.append(types[i]['title'])
            business['categories'].append(res)
        except:
            business['categories'].append('na')
        
    

In [7]:
df=pd.DataFrame(data=business,columns=business.keys())
df

,name,loc,rating,price,area,categories
0,Kozak Ukranian Eatery,"(49.2114757572234, -122.918422905028)",4.5,$,bby,"[Bakeries, Ukrainian]"
1,Milk & Sugar Cafe,"(49.2332961, -123.033879264874)",4.0,$,bby,"[Ice Cream & Frozen Yogurt, Coffee & Tea, Juic..."
2,Dageraad Brewing,"(49.256, -122.9217)",4.5,$,bby,[Breweries]
3,Sugarloop,"(49.2320255365222, -123.026281589961)",4.5,$,bby,"[Desserts, Cafes]"
4,Steel & Oak Brewing,"(49.2020752, -122.9271396)",4.5,$,bby,[Breweries]
...,...,...,...,...,...,...
195,Le Tigre Cuisine,"(49.19358, -123.13076)",4.0,$,van_west,[Food Trucks]
196,Nusa Coffee,"(49.268245, -123.167802)",4.5,$,van_west,"[Coffee & Tea, Sandwiches, Juice Bars & Smooth..."
197,Peqish,"(49.2832832, -123.1147079)",4.0,$,van_west,"[Salad, Sandwiches, Bakeries]"
198,Angus T,"(49.2762326068102, -123.119233623147)",4.0,$,van_west,"[Cafes, Bakeries]"


In [8]:
def filter_loc_by_rat(rat):
    filtered_df=df[df['rating']==rat]
    loc=zip(*list(filtered_df['loc']))
    return loc

In [9]:
google_apikey=os.environ['GOOGLE_API_KEY']
gmap = gmplot.GoogleMapPlotter(49.2827,-123.1207,11,apikey=google_apikey)

gmap.scatter(*filter_loc_by_rat(5),color='red', marker=True)
gmap.scatter(*filter_loc_by_rat(4.5),color='orange',marker=True)
gmap.scatter(*filter_loc_by_rat(4),color='orange',marker=True)
gmap.scatter(*filter_loc_by_rat(3),color='yellow',marker=True)
gmap.scatter(*filter_loc_by_rat(3.5),color='yellow',marker=True)
gmap.draw('map.html')

In [10]:
google_apikey=os.environ['GOOGLE_API_KEY']
gmap = gmplot.GoogleMapPlotter(49.2827,-123.1207,10,apikey=google_apikey)

gmap.heatmap(*zip(*list(df['loc'])))
gmap.draw('heatmap.html')

In [20]:
df['categories'][0]

['Bakeries', 'Ukrainian']

In [14]:
flavors_all=['Asian Fusion','Chinese','French','Hawaiian','Japanese','Indonesian','Korean','Latin American',
 'Malaysian','Mexican',
 'Middle Eastern',
 'Modern European','Russian','Singaporean',
 'Taiwanese',
 'Turkish',
 'Ukrainian',
 'Vietnamese',]

In [15]:
flavors=[]
for word in df['categories'][0]:
    if word in flavors_all:
        flavors.append(word)

In [16]:
flavors

['Ukrainian']

In [58]:
col=[]
for i in range(df['categories'].shape[0]):
    flavors=[]
    for word in df['categories'][i]:
        if word in flavors_all:
            flavors.append(word)
            col.append((i,word))
            break

In [69]:
category=pd.DataFrame(col,columns={0,'flavor'}).set_index(0)
df=df.join(category)

In [72]:
df['flavor'].fillna('cafe',inplace=True)

In [76]:
df.flavor.value_counts()

cafe               155
Chinese             17
Taiwanese            6
Vietnamese           3
Middle Eastern       3
Modern European      2
Malaysian            2
French               2
Japanese             2
Mexican              1
Asian Fusion         1
Ukrainian            1
Russian              1
Korean               1
Turkish              1
Latin American       1
Indonesian           1
Name: flavor, dtype: int64